<a href="https://colab.research.google.com/github/tulaycetin/Veri_Azaltma_Yontemleri/blob/main/%C3%96znitelik_Se%C3%A7imi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**Amaç**: Titanic veri seti üzerinde farklı özellik seçme yöntemlerini (SelectKBest, LassoCV, RFECV) ve SVM sınıflandırıcısını kullanarak “**hayatta kalma**” tahmin performanslarını karşılaştırmak.


In [ ]:
# Gerekli kütüphaneler
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_curve, auc
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest, mutual_info_classif, RFECV
from sklearn.svm import SVC

# **Veri Seti Yükleme ve Ön İnceleme**

In [ ]:
# Hücre 2: Veri setini yükle ve ön inceleme
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

print("Şekil:", df.shape)
print("\nEksik Değer Sayıları:\n", df.isnull().sum())
display(df.head())






# **IterativeImputer:**
**Tüm sayısal sütunlardaki eksik değerleri, diğer sütunlardan öğrenilen istatistikî modellerle (çok değişkenli regresyon) doldurur.**





In [ ]:
# Hücre 3: Sayısal sütunlardaki eksikleri IterativeImputer ile doldur
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
imputer = IterativeImputer(random_state=42)
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])


# **One-Hot Encoding:**
**Kategorik bir özelliği (örn. Sex, Embarked) 0/1 değerleri alan yeni sütunlara dönüştürür. drop_first=True ile bir sütun atılır, böylece lineer bağımlılığın (dummy trap) önüne geçilir.**

In [ ]:
# Hücre 4: One-hot encoding ile kategorikleri sayısala çevir
df = pd.get_dummies(df, drop_first=True)


In [ ]:
print(df.isnull().sum())

**y (“Survived”):** Modelimizin tahmin etmeye çalışacağı etiket.

**X:**Tüm diğer sütunlar, modelin girdisi olacak özellik matrisini oluşturur.

In [ ]:
# Hücre 5: Özellik ve hedef matrisini hazırla
y = df['Survived']
X = df.drop(columns=['Survived'])


# **StandardScaler:**
**Her sütunu ortalaması 0, standart sapması 1 olacak şekilde yeniden ölçeklendirir. SVM gibi mesafe-temelli modellerde önemlidir.**

In [ ]:
# Hücre 6: Özellikleri ölçeklendir
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


# **Özellik Seçme Yöntemleri**

**SelectKBest (mutual_info_classif)**
Karşılıklı bilgi (mutual information) istatistiğine göre en yüksek bilgi değerine sahip ilk 20 özelliği seçer.

**LassoCV (embedded)**
L1 düzenlemeli regresyonla özellik ağırlıklarını küçültür; sıfır olmayan ağırlığa sahip sütunları alır.

**RFECV (wrapper + çapraz doğrulama)**
Tek tek özellikleri kaldırarak (recursive) en iyi doğrulama skorunu veren alt kümesi bulur.

In [ ]:
# Hücre 7: Özellik seçme yöntemleri
# 7.1) SelectKBest (mutual_info)
kbest = SelectKBest(score_func=mutual_info_classif, k=20)
X_kbest = kbest.fit_transform(X_scaled, y)
kbest_cols = X.columns[kbest.get_support()]


# 7.2) LassoCV (embedded)
lasso = LassoCV(cv=5, random_state=42, n_jobs=-1)
lasso.fit(X_scaled, y)
mask_lasso = np.abs(lasso.coef_) > 1e-4
X_lasso = X_scaled[:, mask_lasso]
lasso_cols = X.columns[mask_lasso]



# 7.3) RFECV (wrapper + CV)
svm_lin = SVC(kernel='linear', random_state=42)
rfecv = RFECV(
    estimator=svm_lin,
    step=2,          # her seferinde 2 özellik atlıyor
    cv=StratifiedKFold(5),
    scoring='accuracy',
    n_jobs=-1,
    verbose=2
)
rfecv.fit(X_scaled, y)

X_rfecv = rfecv.transform(X_scaled)
rfecv_cols = X.columns[rfecv.support_]


#**Model Değerlendirme Fonksiyonu**

**Accuracy:** Doğru sınıflandırılanların oranı.

**AUC (Area Under ROC Curve):** Sınıflandırıcının ayırt etme gücünü ölçen eğri altı alan.

**Confusion Matrix ve Classification Report:** Daha ayrıntılı metrikler (precision, recall, f1-score vs.).

In [ ]:
# Hücre 8: Değerlendirme fonksiyonu
def evaluate_svm(X_feat, y, name):
    # Eğitim-test bölme
    X_tr, X_te, y_tr, y_te = train_test_split(
        X_feat, y, test_size=0.2, random_state=42, stratify=y
    )
    # Model eğit ve tahmin
    model = SVC(kernel='rbf', C=0.1, gamma='auto',
                probability=True, class_weight='balanced', random_state=42)
    model.fit(X_tr, y_tr)
    proba = model.predict_proba(X_te)[:, 1]
    pred = (proba >= 0.5).astype(int)

    # Metri̇kler
    acc = accuracy_score(y_te, pred)
    fpr, tpr, _ = roc_curve(y_te, proba)
    roc_auc = auc(fpr, tpr)

    print(f"\n=== {name} ===")
    print(f"Accuracy: {acc:.3f} — AUC: {roc_auc:.3f}")
    print("Confusion Matrix:\n", confusion_matrix(y_te, pred))
    print("Classification Report:\n", classification_report(y_te, pred))

    # ROC eğrisi
    plt.figure(figsize=(5,5))
    plt.plot(fpr, tpr, label=f'{name} (AUC={roc_auc:.2f})')
    plt.plot([0,1],[0,1],'--')
    plt.title(f'ROC Curve — {name}')
    plt.xlabel('FPR'); plt.ylabel('TPR')
    plt.legend()
    plt.show()
    return acc, roc_auc


#**Yöntemlerin Karşılaştırılması**

In [ ]:
# Hücre 9: Tüm yöntemleri değerlendir ve sonuçları karşılaştır
methods = {
    'Orijinal':          X_scaled,
    'SelectKBest(20)':   X_kbest,
    'Lasso':             X_lasso,
    'RFECV':             X_rfecv
}

results = {}
for name, X_m in methods.items():
    results[name] = evaluate_svm(X_m, y, name)

# Hücre 10: Sonuçları tablo olarak göster
res_df = pd.DataFrame(results, index=['Accuracy', 'AUC']).T
display(res_df)
